In [65]:
import pandas as pd
import numpy as np
from datetime import datetime
import pyspark.sql.functions as F
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

### Skip this cell if you already have a chunk of the transactions data.

In [66]:
#file_name = '../data/transactions.csv'
#chunksize = 10 ** 5
#chunk_iter = pd.read_csv(file_name, chunksize=chunksize)
#chunk1 = chunk_iter.next()
#chunk1.to_csv('../data/transactions_chunk1.csv', index=False)

In [67]:
file_name = '../data/transactions_chunk1.csv'
lines = sc.textFile(file_name)

In [68]:
header = lines.first()
lines = lines.filter(lambda row: row != header)

In [69]:
# Functions for parsing the strings from the CSV file

def toIntSafe(inval):
  try:
    return int(inval)
  except ValueError:
    return None

def toTimeSafe(inval):
  try:
    return datetime.strptime(inval, "%Y-%m-%d")
  except ValueError:
    return None

def toLongSafe(inval):
  try:
    return long(inval)
  except ValueError:
    return None

def toFloatSafe(inval):
  try:
    return float(inval)
  except ValueError:
    return None
    
def stringToPost(row):
  r = row.encode('utf8').split(',')
  return Row(
    int(r[0]),         # Don't want this column to be nullable
    toLongSafe(r[1]),
    toLongSafe(r[2]),
    toLongSafe(r[3]),
    toLongSafe(r[4]),
    toLongSafe(r[5]),
    toTimeSafe(r[6]),
    toFloatSafe(r[7]),
    r[8],
    toLongSafe(r[9]),
    toFloatSafe(r[10]))  

In [70]:
transactions_schema = StructType([
    StructField("id", LongType(), False),
    StructField("chain", LongType(), True),
    StructField("dept", LongType(), True),
    StructField("category", LongType(), True),
    StructField("company", LongType(), True),
    StructField("brand", LongType(), True),
    StructField("date", DateType(), True),
    StructField("productsize", DoubleType(), True),
    StructField("purchasemeasure", StringType(), True),
    StructField("purchasequantity", LongType(), True),
    StructField("purchaseamount", DoubleType(), True)
    ])

In [71]:
rowRDD = lines.map(lambda p: stringToPost(p))

In [72]:
transactions = sqlContext.createDataFrame(rowRDD, transactions_schema)

In [73]:
transactions.printSchema()

root
 |-- id: long (nullable = false)
 |-- chain: long (nullable = true)
 |-- dept: long (nullable = true)
 |-- category: long (nullable = true)
 |-- company: long (nullable = true)
 |-- brand: long (nullable = true)
 |-- date: date (nullable = true)
 |-- productsize: double (nullable = true)
 |-- purchasemeasure: string (nullable = true)
 |-- purchasequantity: long (nullable = true)
 |-- purchaseamount: double (nullable = true)



In [74]:
transactions.show()

+-----+-----+----+--------+----------+-----+----------+-----------+---------------+----------------+--------------+
|   id|chain|dept|category|   company|brand|      date|productsize|purchasemeasure|purchasequantity|purchaseamount|
+-----+-----+----+--------+----------+-----+----------+-----------+---------------+----------------+--------------+
|86246|  205|   7|     707|1078778070|12564|2012-03-02|       12.0|             OZ|               1|          7.59|
|86246|  205|  63|    6319| 107654575|17876|2012-03-02|       64.0|             OZ|               1|          1.59|
|86246|  205|  97|    9753|1022027929|    0|2012-03-02|        1.0|             CT|               1|          5.99|
|86246|  205|  25|    2509| 107996777|31373|2012-03-02|       16.0|             OZ|               1|          1.99|
|86246|  205|  55|    5555| 107684070|32094|2012-03-02|       16.0|             OZ|               2|         10.38|
|86246|  205|  97|    9753|1021015020|    0|2012-03-02|        1.0|     

# Training data and testing data

In [75]:
train = pd.read_csv('../data/trainHistory.csv')
offers = pd.read_csv('../data/offers.csv')

In [76]:
# Add department to offers
dept = []
for i in range(offers.shape[0]):
    str_category = str(offers['category'].iloc[i])
    if len(str_category) == 4:
        dept.append(str_category[:2])
    else:
        dept.append(str_category[:1])
        
offers['dept'] = dept
offers['dept'] = pd.to_numeric(offers['dept'])

In [77]:
train = pd.merge(train, offers, how='left', on=['offer'])

In [78]:
# Rename columns
train.columns = ['id', 'chain', 'offer', 'market', 'repeattrips', 'repeater', 'offerdate', 
                 'offer_category', 'quantity', 'offer_company', 'offervalue', 'offer_brand',
                 'offer_dept']

In [79]:
# Reencode the target variable
train['repeater'] = np.where(train['repeater'] == 't', '1', '0')
train['repeater'] = pd.to_numeric(train['repeater'])

In [80]:
train['offerdate'] = train['offerdate'].astype(str)

In [81]:
train_schema = StructType([
    StructField("id", LongType(), False),
    StructField("offer_chain", LongType(), True),
    StructField("offer", LongType(), True),
    StructField("market", LongType(), True),
    StructField("repeattrips", LongType(), True),
    StructField("repeater", IntegerType(), True),
    StructField("offerdate", StringType(), True),
    StructField("offer_category", LongType(), True),
    StructField("quantity", LongType(), True),
    StructField("offer_company", LongType(), True),
    StructField("offervalue", DoubleType(), True),
    StructField("offer_brand", LongType(), True),
    StructField("offer_dept", LongType(), True)
    ])

In [82]:
train_df = sqlContext.createDataFrame(train, train_schema)

In [83]:
test = pd.read_csv('../data/testHistory.csv')
test = pd.merge(test, offers, how='left', on=['offer'])

In [84]:
# Rename columns
test.columns = ['id', 'chain', 'offer', 'market', 'offerdate', 'offer_category', 'quantity', 
                'offer_company', 'offervalue', 'offer_brand', 'offer_dept']

In [85]:
test['offerdate'] = test['offerdate'].astype(str)

In [86]:
test_schema = StructType([
    StructField("id", LongType(), False),
    StructField("offer_chain", LongType(), True),
    StructField("offer", LongType(), True),
    StructField("market", LongType(), True),
    StructField("offerdate", StringType(), True),
    StructField("offer_category", LongType(), True),
    StructField("quantity", LongType(), True),
    StructField("offer_company", LongType(), True),
    StructField("offervalue", DoubleType(), True),
    StructField("offer_brand", LongType(), True),
    StructField("offer_dept", LongType(), True)
    ])

In [87]:
test_df = sqlContext.createDataFrame(test, test_schema)

In [88]:
# This function converts the string cell into a date:
stringToDate = F.udf(lambda x: datetime.strptime(x, '%Y-%m-%d'), DateType())

In [89]:
# Convert offerdate to date type
train_df = train_df.withColumn('offerdate', stringToDate(F.col('offerdate')))
test_df = test_df.withColumn('offerdate', stringToDate(F.col('offerdate')))

In [90]:
train_df = train_df.withColumnRenamed('repeater', 'label')

# Feature Engineering

### Get number of refund transactions and total transactions

In [91]:
id_stats = transactions.select('id', 'purchasequantity', 'purchaseamount',
                              F.when(transactions['purchaseamount'] < 0, 1).otherwise(0).alias('return'))\
                      .withColumn('1', F.lit(1))
id_stats = id_stats.groupBy('id').agg(F.sum('1').alias('total_trans'), 
                                      F.sum('return').alias('total_returns'),
                                      F.avg('purchasequantity').alias('avg_purchasequantity'),
                                      F.avg('purchaseamount').alias('avg_purchaseamount'))
id_stats.show()

+--------+-----------+-------------+--------------------+------------------+
|      id|total_trans|total_returns|avg_purchasequantity|avg_purchaseamount|
+--------+-----------+-------------+--------------------+------------------+
|18470775|        350|            8|                1.34|3.1511428571428635|
|14723452|        755|            5|  1.2264900662251657| 4.073933774834406|
|15738658|         39|            0|   1.205128205128205| 4.780000000000001|
|17552659|        591|            4|  1.5245346869712353| 4.931455160744474|
|12996040|        326|            5|  1.2331288343558282|4.7280981595092095|
|16078766|        966|           54|   1.818840579710145| 5.298498964803263|
|18249735|       1557|           60|  1.2594733461785486|  4.13752087347458|
|14989775|        614|           39|   1.231270358306189|3.8022475570032475|
|15073302|        526|           38|  1.3669201520912548| 4.640133079847899|
|16075389|        591|           32|  1.9593908629441625| 7.553604060913664|

In [92]:
# Rename id column of returns table to avoid ambiguous column names
id_stats = id_stats.withColumnRenamed('id', 'returns_id')

In [93]:
train_df = train_df.join(id_stats, train_df['id'] == id_stats['returns_id'], 'left')
test_df = test_df.join(id_stats, test_df['id'] == id_stats['returns_id'], 'left')

In [94]:
train_df = train_df.select('offer', 'offer_category', 'offer_brand', 'offer_company', 'offervalue', 'offer_dept',
                           'offer_chain',
                           'total_trans', 'total_returns', 'avg_purchasequantity', 'avg_purchaseamount', 'label', 'id')
test_df = test_df.select('offer', 'offer_category', 'offer_brand', 'offer_company', 'offervalue', 'offer_dept',
                         'offer_chain',
                         'total_trans', 'total_returns', 'avg_purchasequantity', 'avg_purchaseamount', 'id')

In [95]:
# Get average value for imputations
avg_total_trans = id_stats.agg(F.avg(F.col('total_trans'))).first()[0]
avg_total_returns = id_stats.agg(F.avg(F.col('total_returns'))).first()[0]
avg_avg_purchasequantity = id_stats.agg(F.avg(F.col('avg_purchasequantity'))).first()[0]
avg_avg_purchaseamount = id_stats.agg(F.avg(F.col('avg_purchaseamount'))).first()[0]

# Impute missing data
impute_dict = {'total_trans': avg_total_trans, 'total_returns': avg_total_returns,
               'avg_purchasequantity': avg_avg_purchasequantity,
               'avg_purchaseamount': avg_avg_purchaseamount}
train_df = train_df.fillna(impute_dict)
test_df = test_df.fillna(impute_dict)

### Get number of transactions that match the offers

In [96]:
train_trans_agg = train_df.join(transactions, 
                            (train_df.id == transactions.id) & (train_df.offer_chain == transactions.chain), 
                            "left") \
                          .select(train_df["id"], "offer_dept", "dept", "offer_company", "company", 
                                  "offer_brand", "brand", "offer_category", "category") \
                          .withColumn("matchDeptCount", F.expr("case when offer_dept = dept then 1 else 0 end")) \
                          .withColumn("matchCategoryCount", F.expr("case when offer_category = category then 1 else 0 end")) \
                          .withColumn("matchCompanyCount", F.expr("case when offer_company = company then 1 else 0 end")) \
                          .withColumn("matchBrandCount", F.expr("case when offer_brand = brand then 1 else 0 end")) \
                          .groupBy("id") \
                          .agg(F.sum("matchDeptCount").alias("matchDeptCount"),
                               F.sum("matchCategoryCount").alias("matchCategoryCount"),
                               F.sum("matchCompanyCount").alias("matchCompanyCount"),
                               F.sum("matchBrandCount").alias("matchBrandCount"))

In [97]:
train_df = train_df.join(train_trans_agg, train_df.id == train_trans_agg.id, "left").drop(train_trans_agg.id)

In [98]:
test_trans_agg = test_df.join(transactions, 
                            (test_df.id == transactions.id) & (test_df.offer_chain == transactions.chain), 
                            "left") \
                          .select(test_df["id"], "offer_dept", "dept", "offer_company", "company", 
                                  "offer_brand", "brand", "offer_category", "category") \
                          .withColumn("matchDeptCount", F.expr("case when offer_dept = dept then 1 else 0 end")) \
                          .withColumn("matchCategoryCount", F.expr("case when offer_category = category then 1 else 0 end")) \
                          .withColumn("matchCompanyCount", F.expr("case when offer_company = company then 1 else 0 end")) \
                          .withColumn("matchBrandCount", F.expr("case when offer_brand = brand then 1 else 0 end")) \
                          .groupBy("id") \
                          .agg(F.sum("matchDeptCount").alias("matchDeptCount"),
                               F.sum("matchCategoryCount").alias("matchCategoryCount"),
                               F.sum("matchCompanyCount").alias("matchCompanyCount"),
                               F.sum("matchBrandCount").alias("matchBrandCount"))

In [99]:
test_df = test_df.join(test_trans_agg, test_df.id == test_trans_agg.id, "left").drop(test_trans_agg.id)

In [100]:
train_df.show()

+-------+--------------+-----------+-------------+----------+----------+-----------+-----------+-------------+--------------------+------------------+-----+---------+--------------+------------------+-----------------+---------------+
|  offer|offer_category|offer_brand|offer_company|offervalue|offer_dept|offer_chain|total_trans|total_returns|avg_purchasequantity|avg_purchaseamount|label|       id|matchDeptCount|matchCategoryCount|matchCompanyCount|matchBrandCount|
+-------+--------------+-----------+-------------+----------+----------+-----------+-----------+-------------+--------------------+------------------+-----+---------+--------------+------------------+-----------------+---------------+
|1197502|          3203|      13474|    106414464|      0.75|        32|         15|       1111|           26|       1.35397034045|      4.4854782917|    0| 93737740|             0|                 0|                0|              0|
|1197502|          3203|      13474|    106414464|      0.75

In [101]:
test_df.show()

+-------+--------------+-----------+-------------+----------+----------+-----------+-----------+-------------+--------------------+------------------+---------+--------------+------------------+-----------------+---------------+
|  offer|offer_category|offer_brand|offer_company|offervalue|offer_dept|offer_chain|total_trans|total_returns|avg_purchasequantity|avg_purchaseamount|       id|matchDeptCount|matchCategoryCount|matchCompanyCount|matchBrandCount|
+-------+--------------+-----------+-------------+----------+----------+-----------+-----------+-------------+--------------------+------------------+---------+--------------+------------------+-----------------+---------------+
|1230218|           706|      26189|    104127141|       1.0|         7|          4|        350|            8|                1.34|3.1511428571428635| 18470775|             0|                 0|                1|              1|
|1221658|          7205|       4294|    103700030|       3.0|        72|         18|


# Create pipeline

In [102]:
# Get all the offers, categories, brands and companies from both training and test
# Then create a StringIndexer to map categorical values to numeric labels
cols_to_index = ['offer', 'offer_category', 'offer_brand', 'offer_company']
indexers = []
for col in cols_to_index:
    union_df = train_df.select(col).union(test_df.select(col))
    indexers.append(StringIndexer().setInputCol(col).setOutputCol(col+'idx').fit(union_df))

In [103]:
# One-hot-encoding of categorical variables
encoders = []
for col in cols_to_index:
    encoders.append(OneHotEncoder().setInputCol(col+'idx').setOutputCol(col+'_encoded').setDropLast(False))

In [104]:
# Assemble features
inputCols = ['offervalue', 'offer_encoded', 'offer_category_encoded', 'offer_brand_encoded', 'offer_company_encoded', 
             'total_trans', 'total_returns', 'avg_purchasequantity', 'avg_purchaseamount',
             'matchDeptCount', 'matchCategoryCount', 'matchCompanyCount', 'matchBrandCount']
assembler = VectorAssembler(inputCols=inputCols, outputCol='features')

In [105]:
# Logistic regression
lr = LogisticRegression(maxIter=10, standardization=True)

In [106]:
# Pipeline
stages = indexers + encoders + [assembler, lr]
pipeline = Pipeline(stages=stages)

# Cross-validation

In [107]:
regularization = [0.0001, 0.001]   # for testing purposes
#regularization = [0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5]
kfolds = 2

paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, regularization) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=kfolds)  # use 3+ folds in practice

In [108]:
train_df.cache()
test_df.cache()

DataFrame[offer: bigint, offer_category: bigint, offer_brand: bigint, offer_company: bigint, offervalue: double, offer_dept: bigint, offer_chain: bigint, total_trans: bigint, total_returns: bigint, avg_purchasequantity: double, avg_purchaseamount: double, id: bigint, matchDeptCount: bigint, matchCategoryCount: bigint, matchCompanyCount: bigint, matchBrandCount: bigint]

In [109]:
cvModel = crossval.fit(train_df)

In [110]:
# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(cvModel.bestModel.stages[-1].coefficients))
print("Intercept: " + str(cvModel.bestModel.stages[-1].intercept))

Coefficients: [-0.042944765501,-0.120829277849,0.0,0.0,0.0,0.211435283644,0.063702827612,0.192685468541,0.0,-0.240028019341,0.0160401214384,0.0,0.12790088634,-0.198890016126,0.282361985977,-0.152731762217,0.0,0.0,0.0399233959859,0.0,0.0,0.0810215871105,0.0,-0.219623550196,-0.123040510053,0.0,-0.109591150214,-0.0843166415836,-0.0661706964977,0.0,0.0,-0.127941743264,-0.0986591356709,-0.12193404927,-0.192676463909,-0.0899420472053,-0.0705863984305,-0.0473132395618,0.0,-0.120829277849,0.0,0.236504958692,0.211435283644,0.0613480473513,-0.232048671675,0.0,0.0,0.117858757622,0.0160401214384,-0.0873034506225,-0.149546005305,-0.118095237047,0.0,-0.213903964144,0.0,0.0,-0.0986591356709,-0.0473132395618,0.0,-0.120829277849,0.0,0.211435283644,0.0613480473513,0.192685468541,-0.232048671675,0.0,0.0,-0.178738832976,0.117858757622,0.0160401214384,-0.0873034506225,0.282361985977,-0.118095237047,0.0,0.0,0.0,-0.0986591356709,0.0,-0.120829277849,-0.0329640371743,0.0,0.211435283644,0.192685468541,-0.232048

# Predicting on training data 

In [111]:
train_pred = cvModel.transform(train_df)

In [112]:
train_pred.select('probability', 'prediction').show()

+--------------------+----------+
|         probability|prediction|
+--------------------+----------+
|[0.82167214173382...|       0.0|
|[0.82167214173382...|       0.0|
|[0.72003730581269...|       0.0|
|[0.72003730581269...|       0.0|
|[0.73328834346035...|       0.0|
|[0.82167214173382...|       0.0|
|[0.82167214173382...|       0.0|
|[0.82167214173382...|       0.0|
|[0.82167214173382...|       0.0|
|[0.82167214173382...|       0.0|
|[0.82167214173382...|       0.0|
|[0.82167214173382...|       0.0|
|[0.82167214173382...|       0.0|
|[0.72003730581269...|       0.0|
|[0.72928071653364...|       0.0|
|[0.82918378511377...|       0.0|
|[0.82918378511377...|       0.0|
|[0.82167214173382...|       0.0|
|[0.82353536839685...|       0.0|
|[0.82167214173382...|       0.0|
+--------------------+----------+
only showing top 20 rows



# Predicting on test data

In [ ]:
test_pred = cvModel.transform(test_df)

In [ ]:
test_pred.select('probability', 'prediction').show()

In [ ]:
test_id = test_df.select('id').toPandas()
test_prob = test_pred.select('probability').toPandas()

In [ ]:
# I didn't know how to access the vector inside the probability column.
# To work around this, I looped through the probability column and used the indexing operations to pull out
# the individual probabilities.
prob = []
for i in range(test_prob.shape[0]):
    prob.append(test_prob['probability'][i][1])
test_prob['repeatProbability'] = prob

In [ ]:
test_final_pred = pd.merge(test_id, test_prob, left_index=True, right_index=True)
test_final_pred.drop('probability', 1, inplace=True)

In [ ]:
test_final_pred.head()

In [ ]:
test_final_pred['repeatProbability'].describe()

In [ ]:
# Write the test file to csv
test_final_pred.to_csv('submission.csv', index=False)